In [ ]:
!pip install xgboost

In [ ]:
!pip install scipy

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
import xgboost as xgb
from sklearn.model_selection import ParameterSampler, train_test_split, GridSearchCV,StratifiedKFold, ParameterGrid
from scipy.stats import uniform, randint
from tqdm.auto import tqdm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc,
    precision_score, recall_score, f1_score, classification_report, balanced_accuracy_score
)
from tqdm.auto import tqdm
from sklearn.svm import SVC
from itertools import product

In [ ]:
df = pd.read_csv('/content/nhanes_merged_data.csv')

In [ ]:
df = df.rename(columns={
    "SEQN": "ID",
    # "MCQ010": 刪掉，直接 drop
    "MCQ035": "asthma",
    "RIDAGEYR": "age",
    "RIAGENDR": "gender",
    "RIDRETH1": "race",
    "DMDEDUC2": "education_adult",
    "DMDEDUC3": "education_child",
    "DMDMARTL": "marital_status",
    "INDFMINC": "family_income",
    "BMXHT": "height_cm",
    "BMXWT": "weight_kg",
    "BMXBMI": "bmi",
    "BMXWAIST": "waist_cm",
    "BMXTRI": "triceps_skinfold_mm",
    "BMXSUB": "subscapular_skinfold_mm",
    "SMQ020": "ever_smoked",
    "SMQ040": "current_smoking",
    "SLQ050": "sleep_difficulty",
    "LBXIGE": "total_IgE",
    "LBXID1": "IgE_dustmite",
    "LBXID2": "IgE_housemite",
    "LBXIE1": "IgE_cat",
    "LBXIE5": "IgE_dog",
    "LBXCOT": "cotinine",
    "LBDVIDMS": "vitaminD",
    "LBXBCD": "blood_cadmium",
    "LBXBPB": "blood_lead",
    "LBXTHG": "blood_mercury",
    "URXUAS": "urine_total_arsenic",
    "URXUHG": "urine_inorganic_mercury",
    "RDQ070": "wheezing",
    "RDQ100": "wheezing_exercise",
    "RDQ090": "wheezing_sleep",
    "SMD410": "household_smoker"
})



In [ ]:
df.loc[df["wheezing_sleep"] < 1e-10, "wheezing_sleep"] = 0
df["wheezing_exercise"] = df["wheezing_exercise"].fillna(0)
df["wheezing_sleep"] = df["wheezing_sleep"].fillna(2)
df = df.replace({77: np.nan, 99: np.nan})
columns_to_clean = [
    "wheezing", "education_adult", "MCQ010", "household_smoker",
    "sleep_difficulty", "ever_smoked", "current_smoking",
    "wheezing_exercise", "wheezing_sleep"
]
df[columns_to_clean] = df[columns_to_clean].replace({7: np.nan, 9: np.nan})

df = df[df["MCQ010"].notna()]

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9809 entries, 0 to 9821
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       9809 non-null   float64
 1   MCQ010                   9809 non-null   float64
 2   asthma                   1389 non-null   float64
 3   age                      9771 non-null   float64
 4   gender                   9809 non-null   float64
 5   race                     9809 non-null   float64
 6   education_adult          4963 non-null   float64
 7   education_child          3428 non-null   float64
 8   marital_status           6680 non-null   float64
 9   family_income            9501 non-null   float64
 10  height_cm                8953 non-null   float64
 11  weight_kg                9304 non-null   float64
 12  bmi                      8940 non-null   float64
 13  waist_cm                 8613 non-null   float64
 14  triceps_skinfold_mm      8481

#### NHANES原始代碼：

1 = Yes (現在有氣喘)=>1/Other = 0

In [ ]:
df["asthma"] = (df["asthma"] == 1).astype(int)

/tmp/ipython-input-1927492071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["asthma"] = (df["asthma"] == 1).astype(int)


In [ ]:
print(sum(df["asthma"]==0))
print(sum(df["asthma"]==1))

8936
873


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9809 entries, 0 to 9821
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       9809 non-null   float64
 1   MCQ010                   9809 non-null   float64
 2   asthma                   9809 non-null   int64  
 3   age                      9771 non-null   float64
 4   gender                   9809 non-null   float64
 5   race                     9809 non-null   float64
 6   education_adult          4963 non-null   float64
 7   education_child          3428 non-null   float64
 8   marital_status           6680 non-null   float64
 9   family_income            9501 non-null   float64
 10  height_cm                8953 non-null   float64
 11  weight_kg                9304 non-null   float64
 12  bmi                      8940 non-null   float64
 13  waist_cm                 8613 non-null   float64
 14  triceps_skinfold_mm      8481

In [ ]:
seed = 2025

In [ ]:
# 假設 'Diagnosis' 是目標欄位（請依你資料中的實際欄位名調整）
X = df.drop(columns=['ID',"MCQ010", "asthma"])
y = df['asthma']

# Step 1: 先切出 20% 的測試資料
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=seed
)

# 檢查比例
print(f"Train: {len(X_train)} samples")
print(f"Test: {len(X_test)} samples")


Train: 7847 samples
Test: 1962 samples


In [ ]:
from collections import Counter
def print_class_distribution(name, y):
    counter = Counter(y)
    total = len(y)
    print(f"{name} Set Class Distribution:")
    for cls, count in counter.items():
        print(f"Class {cls}: {count} samples ({count / total:.2%})")
    print()

# 顯示類別比例
print_class_distribution("Train", y_train)
print_class_distribution("Test", y_test)

Train Set Class Distribution:
Class 0: 7149 samples (91.10%)
Class 1: 698 samples (8.90%)

Test Set Class Distribution:
Class 0: 1787 samples (91.08%)
Class 1: 175 samples (8.92%)



# Data Preprocessing Pipeline


In [ ]:
# Drop缺失值太多欄位
X_train.drop(columns=["urine_inorganic_mercury", "urine_total_arsenic"], inplace=True)
X_test.drop(columns=["urine_inorganic_mercury", "urine_total_arsenic"], inplace=True)

# Education要先合併
def map_education_child(x):
    if pd.isnull(x):
        return np.nan
    if x in [0, 1, 2, 3, 4, 5, 55, 6, 7, 8, 66]:  # 國小以下 + 國小
        return 1
    elif x in [9, 10, 11, 12]:  # 國中
        return 2
    elif x in [13, 14]:  # 高中職
        return 3
    elif x == 15:  # 大學
        return 4
    else:
        return np.nan

def map_education_adult(x):
    if pd.isnull(x):
        return np.nan
    if x == 1:
        return 1  # 小學
    elif x == 2:
        return 2  # 國中
    elif x == 3:
        return 3  # 高中
    elif x == 4:
        return 4  # 大學
    elif x == 5:
        return 5  # 研究所
    else:
        return np.nan
def process_education(df):
    # Apply mapping
    df["education_child_level"] = df["education_child"].apply(map_education_child)
    df["education_adult_level"] = df["education_adult"].apply(map_education_adult)

    # Merge into single education column
    df["education"] = np.where(
        df["age"] <= 20,
        df["education_child_level"],
        df["education_adult_level"]
    )

    # Drop original and intermediate columns
    df = df.drop(columns=[
        "education_child",
        "education_adult",
        "education_child_level",
        "education_adult_level"
    ])

    return df

X_train = process_education(X_train)
X_test = process_education(X_test)

## height/weight/bmi
def fill_height_weight_bmi_partial(df):
    h = df["height_cm"]
    w = df["weight_kg"]
    b = df["bmi"]

    # 補 BMI：有身高與體重，但 BMI 缺
    mask_bmi_missing = b.isna() & h.notna() & w.notna()
    df.loc[mask_bmi_missing, "bmi"] = df.loc[mask_bmi_missing, "weight_kg"] / (df.loc[mask_bmi_missing, "height_cm"] / 100) ** 2

    # 補體重：有身高與 BMI，但體重缺
    mask_weight_missing = w.isna() & h.notna() & b.notna()
    df.loc[mask_weight_missing, "weight_kg"] = df.loc[mask_weight_missing, "bmi"] * (df.loc[mask_weight_missing, "height_cm"] / 100) ** 2

    # 補身高：有體重與 BMI，但身高缺
    mask_height_missing = h.isna() & w.notna() & b.notna()
    df.loc[mask_height_missing, "height_cm"] = np.sqrt(df.loc[mask_height_missing, "weight_kg"] / df.loc[mask_height_missing, "bmi"]) * 100

    return df

def fill_remaining_height_weight_bmi(df, median_height, median_weight):
    # === 情況 1: height_weight_missing (身高 + 體重缺，BMI 有值) ===
    mask_height_weight_missing = df["height_cm"].isna() & df["weight_kg"].isna() & df["bmi"].notna()
    df.loc[mask_height_weight_missing, "weight_kg"] = median_weight
    # 使用填補後的體重 & 現有 BMI 算出身高
    df.loc[mask_height_weight_missing, "height_cm"] = np.sqrt(
        df.loc[mask_height_weight_missing, "weight_kg"] / df.loc[mask_height_weight_missing, "bmi"]
    ) * 100

    # === 情況 2: height_bmi_missing (身高 + BMI 缺) ===
    mask_height_bmi_missing = df["height_cm"].isna() & df["weight_kg"].notna() & df["bmi"].isna()
    df.loc[mask_height_bmi_missing, "height_cm"] = median_height
    # 使用補完的身高 & 現有體重計算 BMI
    df.loc[mask_height_bmi_missing, "bmi"] = df.loc[mask_height_bmi_missing, "weight_kg"] / (
        (df.loc[mask_height_bmi_missing, "height_cm"] / 100) ** 2
    )

    # === 情況 3: weight_bmi_missing (體重 + BMI 缺) ===
    mask_weight_bmi_missing = df["weight_kg"].isna() & df["height_cm"].notna() & df["bmi"].isna()
    df.loc[mask_weight_bmi_missing, "weight_kg"] = median_weight
    # 使用補完的體重 & 現有身高計算 BMI
    df.loc[mask_weight_bmi_missing, "bmi"] = df.loc[mask_weight_bmi_missing, "weight_kg"] / (
        (df.loc[mask_weight_bmi_missing, "height_cm"] / 100) ** 2
    )

    # === 情況 4: all_missing (全部缺) ===
    mask_all_missing = df["height_cm"].isna() & df["weight_kg"].isna() & df["bmi"].isna()
    df.loc[mask_all_missing, "height_cm"] = median_height
    df.loc[mask_all_missing, "weight_kg"] = median_weight
    df.loc[mask_all_missing, "bmi"] = df.loc[mask_all_missing, "weight_kg"] / (
        (df.loc[mask_all_missing, "height_cm"] / 100) ** 2
    )

    return df


# 1. 先補 X_train 的 height/weight/bmi 的可推情況（不補中位數）
X_train = fill_height_weight_bmi_partial(X_train)

# 2. 現在再算中位數
train_median_height = X_train["height_cm"].median()
train_median_weight = X_train["weight_kg"].median()

# 3. 補剩下缺的欄位
X_train = fill_remaining_height_weight_bmi(X_train, train_median_height, train_median_weight)

# 4. 用同樣中位數去補 X_test
X_test = fill_height_weight_bmi_partial(X_test)
X_test = fill_remaining_height_weight_bmi(X_test, train_median_height, train_median_weight)

## 中位數填補
# 1. 定義要補的欄位
median_fill_columns = [
    "age", "waist_cm", "triceps_skinfold_mm", "subscapular_skinfold_mm",
    "cotinine", "vitaminD", "total_IgE", "IgE_dustmite", "IgE_housemite",
    "IgE_cat", "IgE_dog", "blood_cadmium", "blood_lead", "blood_mercury"
]

# 2. 根據 X_train 計算中位數
train_medians = X_train[median_fill_columns].median()

# 3. 使用中位數填補缺失
X_train[median_fill_columns] = X_train[median_fill_columns].fillna(train_medians)
X_test[median_fill_columns] = X_test[median_fill_columns].fillna(train_medians)

## 眾數填補
X_train.loc[X_train["age"] < 14, "marital_status"] = X_train.loc[X_train["age"] < 14, "marital_status"].fillna(5)
X_test.loc[X_test["age"] < 14, "marital_status"] = X_test.loc[X_test["age"] < 14, "marital_status"].fillna(5)

X_train.loc[X_train["age"] < 20, "ever_smoked"] = X_train.loc[X_train["age"] < 20, "ever_smoked"].fillna(0)
X_test.loc[X_test["age"] < 20, "ever_smoked"] = X_test.loc[X_test["age"] < 20, "ever_smoked"].fillna(0)

X_train.loc[X_train["age"] < 20, "current_smoking"] = X_train.loc[X_train["age"] < 20, "current_smoking"].fillna(0)
X_test.loc[X_test["age"] < 20, "current_smoking"] = X_test.loc[X_test["age"] < 20, "current_smoking"].fillna(0)

X_train.loc[X_train["age"] < 16, "sleep_difficulty"] = X_train.loc[X_train["age"] < 16, "sleep_difficulty"].fillna(0)
X_test.loc[X_test["age"] < 16, "sleep_difficulty"] = X_test.loc[X_test["age"] < 16, "sleep_difficulty"].fillna(0)

X_train["education"] = X_train["education"].fillna(6)

X_test["education"] = X_test["education"].fillna(6)

X_train["family_income"] = X_train["family_income"].fillna(14)
X_test["family_income"] = X_test["family_income"].fillna(14)

X_train["marital_status"] = X_train["marital_status"].fillna(7)
X_test["marital_status"] = X_test["marital_status"].fillna(7)

X_train["current_smoking"] = X_train["current_smoking"].fillna(4)
X_test["current_smoking"] = X_test["current_smoking"].fillna(4)


# 其他類別型欄位（排除 wheezing_sleep）
other_categorical = [
    "ever_smoked", "sleep_difficulty",
    "wheezing", "wheezing_sleep", "wheezing_exercise", "household_smoker"
]

# 其他欄位的缺失值補 0
X_train[other_categorical] = X_train[other_categorical].fillna(3)
X_test[other_categorical] = X_test[other_categorical].fillna(3)

In [ ]:
def missing_report(df):
    missing_count = df.isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    report = pd.DataFrame({
        'MissingCount': missing_count,
        'MissingPercent': missing_pct
    }).sort_values(by='MissingPercent', ascending=False)
    report = report[report['MissingPercent'] > 0]
    return report
print("\n=== Training Set Missing Value Report ===")
print(missing_report(X_train))


print("\n=== Test Set Missing Value Report ===")
print(missing_report(X_test))


=== Training Set Missing Value Report ===
Empty DataFrame
Columns: [MissingCount, MissingPercent]
Index: []

=== Test Set Missing Value Report ===
Empty DataFrame
Columns: [MissingCount, MissingPercent]
Index: []


In [ ]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 7847 entries, 6794 to 3674
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      7847 non-null   float64
 1   gender                   7847 non-null   float64
 2   race                     7847 non-null   float64
 3   marital_status           7847 non-null   float64
 4   family_income            7847 non-null   float64
 5   height_cm                7847 non-null   float64
 6   weight_kg                7847 non-null   float64
 7   bmi                      7847 non-null   float64
 8   waist_cm                 7847 non-null   float64
 9   triceps_skinfold_mm      7847 non-null   float64
 10  subscapular_skinfold_mm  7847 non-null   float64
 11  ever_smoked              7847 non-null   float64
 12  current_smoking          7847 non-null   float64
 13  sleep_difficulty         7847 non-null   float64
 14  total_IgE                7

## Preprocessing


In [ ]:
minmax_columns = [
    "age", "height_cm", "weight_kg", "bmi",
    "waist_cm", "triceps_skinfold_mm", "subscapular_skinfold_mm",
    "cotinine", "vitaminD", "total_IgE", "IgE_dustmite", "IgE_housemite",
    "IgE_cat", "IgE_dog", "blood_cadmium", "blood_lead", "blood_mercury"
]
onehot_columns = [
    "gender", "race", "marital_status", "education", "family_income",
    "ever_smoked", "current_smoking", "sleep_difficulty",
    "wheezing", "wheezing_sleep", "wheezing_exercise", "household_smoker"
]

# MinMax normalization
scaler = MinMaxScaler()
X_train[minmax_columns] = scaler.fit_transform(X_train[minmax_columns])
X_test[minmax_columns] = scaler.transform(X_test[minmax_columns])

for col in onehot_columns:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

# One-Hot Encoding
X_train = pd.get_dummies(X_train, columns=onehot_columns)
X_test = pd.get_dummies(X_test, columns=onehot_columns)

# 補齊測試集欄位
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 確保所有 one-hot 欄位為 int 而非 bool
X_train = X_train.astype({col: int for col in X_train.select_dtypes(bool).columns})
X_test = X_test.astype({col: int for col in X_test.select_dtypes(bool).columns})
print(X_train.head())
print(X_test.head())

           age  height_cm  weight_kg       bmi  waist_cm  triceps_skinfold_mm  \
6794  0.011905   0.089330   0.011836  0.091868  0.022255             0.166667   
1357  0.035714   0.170389   0.023672  0.104045  0.080119             0.128571   
5009  0.214286   0.939620   0.202863  0.147963  0.307122             0.147619   
7040  0.785714   0.681555   0.224883  0.243977  0.336053             0.266667   
884   0.142857   0.661704   0.210295  0.235720  0.341246             0.747619   

      subscapular_skinfold_mm  total_IgE  IgE_dustmite  IgE_housemite  ...  \
6794                 0.150485   0.000491       0.00000        0.00000  ...   
1357                 0.126214   0.003536       0.00000        0.00000  ...   
5009                 0.218447   0.003015       0.00000        0.00000  ...   
7040                 0.250000   0.055466       0.00084        0.00178  ...   
884                  0.623786   0.003028       0.00000        0.00000  ...   

      wheezing_sleep_1.0  wheezing_sleep_2.0

In [ ]:
x = X_train.values
y = y_train.values

## balanced-accuracy

#### Random Forest(balanced_weight)

In [ ]:
def compute_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    balanced_acc = (sensitivity + specificity) / 2
    return balanced_acc, accuracy, sensitivity, specificity

In [ ]:
# 參數網格
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 15, 20],
    "min_samples_split": [2, 3, 4, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None],
    "max_leaf_nodes": [10, 20, 30, 40]
}


param_list = list(ParameterGrid(param_grid))

print(f"Total combinations: {len(param_list)}")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

best_score = 0
best_params = None
best_metrics = None

print("Running Grid Search with 5-Fold CV (train/val metrics)...\n")

for i, param in enumerate(param_list):
    fold_train_bal_acc = []
    fold_val_bal_acc = []

    fold_train_acc = []
    fold_val_acc = []

    fold_train_sens = []
    fold_val_sens = []

    fold_train_spec = []
    fold_val_spec = []

    print(f"Param Set {i+1}/{len(param_list)}: {param}")

    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(x, y)):
        x_train_fold, y_train_fold = x[train_idx], y[train_idx]
        x_val_fold, y_val_fold = x[val_idx], y[val_idx]

        clf = RandomForestClassifier(
            **param,
            random_state=seed,
            n_jobs=-1,
            class_weight='balanced'
        )
        clf.fit(x_train_fold, y_train_fold)

        # 預測
        y_train_pred = clf.predict(x_train_fold)
        y_val_pred = clf.predict(x_val_fold)

        # 計算指標
        train_bal_acc, train_acc, train_sens, train_spec = compute_metrics(y_train_fold, y_train_pred)
        val_bal_acc, val_acc, val_sens, val_spec = compute_metrics(y_val_fold, y_val_pred)

        # 累積
        fold_train_bal_acc.append(train_bal_acc)
        fold_val_bal_acc.append(val_bal_acc)

        fold_train_acc.append(train_acc)
        fold_val_acc.append(val_acc)

        fold_train_sens.append(train_sens)
        fold_val_sens.append(val_sens)

        fold_train_spec.append(train_spec)
        fold_val_spec.append(val_spec)

        # ✅ 印這個 fold 的結果
        #print(f"  Fold {fold_idx+1}:")
        #print(f"    [TRAIN] Balanced Acc = {train_bal_acc:.4f} | Acc = {train_acc:.4f} | Sens = {train_sens:.4f} | Spec = {train_spec:.4f}")
        #print(f"    [VALID] Balanced Acc = {val_bal_acc:.4f} | Acc = {val_acc:.4f} | Sens = {val_sens:.4f} | Spec = {val_spec:.4f}")

    # 平均
    avg_train_bal_acc = np.mean(fold_train_bal_acc)
    avg_val_bal_acc = np.mean(fold_val_bal_acc)

    avg_train_acc = np.mean(fold_train_acc)
    avg_val_acc = np.mean(fold_val_acc)

    avg_train_sens = np.mean(fold_train_sens)
    avg_val_sens = np.mean(fold_val_sens)

    avg_train_spec = np.mean(fold_train_spec)
    avg_val_spec = np.mean(fold_val_spec)

    print(f"Mean over 5 folds:")
    print(f"   [TRAIN] Balanced Acc = {avg_train_bal_acc:.4f} | Acc = {avg_train_acc:.4f} | Sens = {avg_train_sens:.4f} | Spec = {avg_train_spec:.4f}")
    print(f"   [VALID] Balanced Acc = {avg_val_bal_acc:.4f} | Acc = {avg_val_acc:.4f} | Sens = {avg_val_sens:.4f} | Spec = {avg_val_spec:.4f}")

    if avg_val_bal_acc > best_score:
        best_score = avg_val_bal_acc
        best_params = param
        best_metrics = {
            "Balanced Accuracy": avg_val_bal_acc,
            "Accuracy": avg_val_acc,
            "Sensitivity": avg_val_sens,
            "Specificity": avg_val_spec,
            "Train Balanced Accuracy": avg_train_bal_acc,
            "Train Accuracy": avg_train_acc,
            "Train Sensitivity": avg_train_sens,
            "Train Specificity": avg_train_spec
        }

print("\n Best Random Search Result:")
print("Best Params:", best_params)
print(f"Best Balanced Accuracy: {best_score:.4f}")
print(f"Best Accuracy:           {best_metrics['Accuracy']:.4f}")
print(f"Best Sensitivity:        {best_metrics['Sensitivity']:.4f}")
print(f"Best Specificity:        {best_metrics['Specificity']:.4f}")
print(f"Best Training Balanced Accuracy:   {best_metrics['Train Balanced Accuracy']:.4f}")
print(f"Best Training Accuracy:            {best_metrics['Train Accuracy']:.4f}")
print(f"Best Training Sensitivity:         {best_metrics['Train Sensitivity']:.4f}")
print(f"Best Training Specificity:         {best_metrics['Train Specificity']:.4f}")


串流輸出內容已截斷至最後 5000 行。
   [VALID] Balanced Acc = 0.7951 | Acc = 0.8752 | Sens = 0.6976 | Spec = 0.8926
Param Set 914/2160: {'max_depth': 10, 'max_features': None, 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Mean over 5 folds:
   [TRAIN] Balanced Acc = 0.8176 | Acc = 0.8826 | Sens = 0.7385 | Spec = 0.8966
   [VALID] Balanced Acc = 0.7980 | Acc = 0.8759 | Sens = 0.7033 | Spec = 0.8927
Param Set 915/2160: {'max_depth': 10, 'max_features': None, 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Mean over 5 folds:
   [TRAIN] Balanced Acc = 0.8181 | Acc = 0.8826 | Sens = 0.7396 | Spec = 0.8966
   [VALID] Balanced Acc = 0.7989 | Acc = 0.8764 | Sens = 0.7047 | Spec = 0.8931
Param Set 916/2160: {'max_depth': 10, 'max_features': None, 'max_leaf_nodes': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Mean over 5 folds:
   [TRAIN] Balanced Acc = 0.8166 | Acc = 0.8829 | Sens = 0.7360 | Spec

##### Test

In [ ]:
print("\nTraining final model on full training data with best parameters...")
final_model = RandomForestClassifier(
    **best_params,
    random_state=seed,
    n_jobs=-1,
    class_weight='balanced'
)
final_model.fit(x, y)

# 預測 Test Set
y_test_proba = final_model.predict_proba(X_test.values)[:, 1]
y_test_pred = (y_test_proba >= 0.5).astype(int)

cm = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm.ravel()

# 指標計算
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
accuracy = (tp + tn) / (tp + tn + fp + fn)
balanced_acc = (sensitivity + specificity) / 2
auc = roc_auc_score(y_test, y_test_proba)
f1 = f1_score(y_test, y_test_pred, zero_division=0)

# 轉成 DataFrame (Wikipedia 標準)
confusion_df = pd.DataFrame(
    data=[[tp, fn], [fp, tn]],
    index=["Actual Positive", "Actual Negative"],
    columns=["Predicted Positive", "Predicted Negative"]
)

print("\n✅ Confusion Matrix (DataFrame, Wikipedia format):")
print(confusion_df)

# 顯示結果
print(f"Sensitivity (Recall, TPR): {sensitivity:.4f}")
print(f"Specificity (TNR):         {specificity:.4f}")
print(f"Accuracy:                  {accuracy:.4f}")
print(f"Balanced Accuracy:         {balanced_acc:.4f}")
print(f"AUC:                       {auc:.4f}")
print(f"F1 Score:         {f1:.4f}")


Training final model on full training data with best parameters...

✅ Confusion Matrix (DataFrame, Wikipedia format):
                 Predicted Positive  Predicted Negative
Actual Positive                 112                  63
Actual Negative                 143                1644
Sensitivity (Recall, TPR): 0.6400
Specificity (TNR):         0.9200
Accuracy:                  0.8950
Balanced Accuracy:         0.7800
AUC:                       0.8565
F1 Score:         0.5209


#### Logistic Regression(balanced-weight)

In [ ]:
C_list = [2 ** i for i in range(-10, 11)]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

best_score = 0
best_params = None
best_metrics = None

print("Running Grid Search over C with 5-Fold Stratified CV...\n")

for C in tqdm(C_list):
    fold_train_bal_acc = []
    fold_train_acc = []
    fold_train_sens = []
    fold_train_spec = []

    fold_val_bal_acc = []
    fold_val_acc = []
    fold_val_sens = []
    fold_val_spec = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(x, y), 1):
        x_train_fold, y_train_fold = x[train_idx], y[train_idx]
        x_val_fold, y_val_fold = x[val_idx], y[val_idx]

        clf = LogisticRegression(
            C=C,
            penalty='l2',
            max_iter=1000,
            solver='lbfgs',
            class_weight='balanced',
            random_state=seed
        )
        clf.fit(x_train_fold, y_train_fold)

        # --- Train
        y_train_pred = clf.predict(x_train_fold)
        tn, fp, fn, tp = confusion_matrix(y_train_fold, y_train_pred).ravel()
        train_sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        train_spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        train_acc = (tp + tn) / (tp + tn + fp + fn)
        train_bal_acc = (train_sens + train_spec) / 2

        fold_train_bal_acc.append(train_bal_acc)
        fold_train_acc.append(train_acc)
        fold_train_sens.append(train_sens)
        fold_train_spec.append(train_spec)

        # --- Validation
        y_val_pred = clf.predict(x_val_fold)
        tn, fp, fn, tp = confusion_matrix(y_val_fold, y_val_pred).ravel()
        val_sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        val_spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        val_acc = (tp + tn) / (tp + tn + fp + fn)
        val_bal_acc = (val_sens + val_spec) / 2

        fold_val_bal_acc.append(val_bal_acc)
        fold_val_acc.append(val_acc)
        fold_val_sens.append(val_sens)
        fold_val_spec.append(val_spec)

        #print(f"C={C:<10} Fold {fold}:")
        #print(f"  Train - BalAcc: {train_bal_acc:.4f} Acc: {train_acc:.4f} Sens: {train_sens:.4f} Spec: {train_spec:.4f}")
        #print(f"  Val   - BalAcc: {val_bal_acc:.4f} Acc: {val_acc:.4f} Sens: {val_sens:.4f} Spec: {val_spec:.4f}")

    # === Mean over folds
    avg_train_bal_acc = np.mean(fold_train_bal_acc)
    avg_train_acc = np.mean(fold_train_acc)
    avg_train_sens = np.mean(fold_train_sens)
    avg_train_spec = np.mean(fold_train_spec)

    avg_val_bal_acc = np.mean(fold_val_bal_acc)
    avg_val_acc = np.mean(fold_val_acc)
    avg_val_sens = np.mean(fold_val_sens)
    avg_val_spec = np.mean(fold_val_spec)

    print(f"\nMean over folds for C={C}:")
    print(f"  Train - BalAcc: {avg_train_bal_acc:.4f} Acc: {avg_train_acc:.4f} Sens: {avg_train_sens:.4f} Spec: {avg_train_spec:.4f}")
    print(f"  Val   - BalAcc: {avg_val_bal_acc:.4f} Acc: {avg_val_acc:.4f} Sens: {avg_val_sens:.4f} Spec: {avg_val_spec:.4f}\n")

    # --- Best tracking
    if avg_val_bal_acc > best_score:
        best_score = avg_val_bal_acc
        best_params = C
        best_metrics = {
            "Balanced Accuracy": avg_val_bal_acc,
            "Accuracy": avg_val_acc,
            "Sensitivity": avg_val_sens,
            "Specificity": avg_val_spec,
            "Train Balanced Accuracy": avg_train_bal_acc,
            "Train Accuracy": avg_train_acc,
            "Train Sensitivity": avg_train_sens,
            "Train Specificity": avg_train_spec,
        }

print("\n Best Grid Search Result:")
print(f"Best C: {best_params}")
print(f"Best Validation Balanced Accuracy: {best_metrics['Balanced Accuracy']:.4f}")
print(f"Best Validation Accuracy:          {best_metrics['Accuracy']:.4f}")
print(f"Best Validation Sensitivity:       {best_metrics['Sensitivity']:.4f}")
print(f"Best Validation Specificity:       {best_metrics['Specificity']:.4f}")
print(f"Best Training Balanced Accuracy:   {best_metrics['Train Balanced Accuracy']:.4f}")
print(f"Best Training Accuracy:            {best_metrics['Train Accuracy']:.4f}")
print(f"Best Training Sensitivity:         {best_metrics['Train Sensitivity']:.4f}")
print(f"Best Training Specificity:         {best_metrics['Train Specificity']:.4f}")


Running Grid Search over C with 5-Fold Stratified CV...



  0%|          | 0/21 [00:00<?, ?it/s]


Mean over folds for C=0.0009765625:
  Train - BalAcc: 0.7989 Acc: 0.8951 Sens: 0.6819 Spec: 0.9159
  Val   - BalAcc: 0.7989 Acc: 0.8951 Sens: 0.6818 Spec: 0.9159


Mean over folds for C=0.001953125:
  Train - BalAcc: 0.7989 Acc: 0.8951 Sens: 0.6819 Spec: 0.9159
  Val   - BalAcc: 0.7989 Acc: 0.8951 Sens: 0.6818 Spec: 0.9159


Mean over folds for C=0.00390625:
  Train - BalAcc: 0.7989 Acc: 0.8951 Sens: 0.6819 Spec: 0.9159
  Val   - BalAcc: 0.7989 Acc: 0.8951 Sens: 0.6818 Spec: 0.9159


Mean over folds for C=0.0078125:
  Train - BalAcc: 0.7991 Acc: 0.8952 Sens: 0.6823 Spec: 0.9159
  Val   - BalAcc: 0.7989 Acc: 0.8951 Sens: 0.6818 Spec: 0.9159


Mean over folds for C=0.015625:
  Train - BalAcc: 0.7991 Acc: 0.8948 Sens: 0.6827 Spec: 0.9155
  Val   - BalAcc: 0.7987 Acc: 0.8949 Sens: 0.6818 Spec: 0.9157


Mean over folds for C=0.03125:
  Train - BalAcc: 0.8002 Acc: 0.8942 Sens: 0.6859 Spec: 0.9145
  Val   - BalAcc: 0.7985 Acc: 0.8932 Sens: 0.6833 Spec: 0.9137


Mean over folds for C=0.0625:


##### Test

In [ ]:
# 取得最佳參數
final_model = LogisticRegression(
    C=best_params,
    penalty='l2',
    max_iter=1000,
    solver='lbfgs',
    class_weight='balanced',
    random_state=seed
)
final_model.fit(x, y)


# 預測
y_test_proba = final_model.predict_proba(X_test.values)[:, 1]
y_test_pred = (y_test_proba >= 0.5).astype(int)


# 混淆矩陣
from sklearn.metrics import confusion_matrix, roc_auc_score

cm = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm.ravel()

# 指標計算
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
accuracy = (tp + tn) / (tp + tn + fp + fn)
balanced_acc = (sensitivity + specificity) / 2
auc = roc_auc_score(y_test, y_test_proba)
f1 = f1_score(y_test, y_test_pred, zero_division=0)

# 轉成 DataFrame (Wikipedia 標準)
confusion_df = pd.DataFrame(
    data=[[tp, fn], [fp, tn]],
    index=["Actual Positive", "Actual Negative"],
    columns=["Predicted Positive", "Predicted Negative"]
)

print("\n✅ Confusion Matrix (DataFrame, Wikipedia format):")
print(confusion_df)

# 顯示結果
print(f"Sensitivity (Recall, TPR): {sensitivity:.4f}")
print(f"Specificity (TNR):         {specificity:.4f}")
print(f"Accuracy:                  {accuracy:.4f}")
print(f"Balanced Accuracy:         {balanced_acc:.4f}")
print(f"AUC:                       {auc:.4f}")
print(f"F1 Score:         {f1:.4f}")


✅ Confusion Matrix (DataFrame, Wikipedia format):
                 Predicted Positive  Predicted Negative
Actual Positive                 116                  59
Actual Negative                 163                1624
Sensitivity (Recall, TPR): 0.6629
Specificity (TNR):         0.9088
Accuracy:                  0.8869
Balanced Accuracy:         0.7858
AUC:                       0.8503
F1 Score:         0.5110


#### SVM(balanced-weight)

In [ ]:
# 定義超參數範圍
C_list = [2 ** i for i in range(-10, 11)]
gamma_list = [2 ** i for i in range(-10, 11)]
param_grid = list(product(C_list, gamma_list))

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

best_score = 0
best_params = None
best_metrics = None

from tqdm import tqdm
print("Running Grid Search over C and gamma with 5-Fold Stratified CV...\n")

for C, gamma in tqdm(param_grid):
    fold_train_bal_acc = []
    fold_train_acc = []
    fold_train_sens = []
    fold_train_spec = []

    fold_val_bal_acc = []
    fold_val_acc = []
    fold_val_sens = []
    fold_val_spec = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(x, y), 1):
        x_train_fold, y_train_fold = x[train_idx], y[train_idx]
        x_val_fold, y_val_fold = x[val_idx], y[val_idx]

        clf = SVC(
            C=C,
            gamma=gamma,
            kernel='rbf',
            class_weight='balanced',
            probability=True,
            random_state=seed
        )
        clf.fit(x_train_fold, y_train_fold)

        # --- Train
        y_train_pred = clf.predict(x_train_fold)
        tn, fp, fn, tp = confusion_matrix(y_train_fold, y_train_pred).ravel()
        train_sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        train_spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        train_acc = (tp + tn) / (tp + tn + fp + fn)
        train_bal_acc = (train_sens + train_spec) / 2

        fold_train_bal_acc.append(train_bal_acc)
        fold_train_acc.append(train_acc)
        fold_train_sens.append(train_sens)
        fold_train_spec.append(train_spec)

        # --- Validation
        y_val_pred = clf.predict(x_val_fold)
        tn, fp, fn, tp = confusion_matrix(y_val_fold, y_val_pred).ravel()
        val_sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        val_spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        val_acc = (tp + tn) / (tp + tn + fp + fn)
        val_bal_acc = (val_sens + val_spec) / 2

        fold_val_bal_acc.append(val_bal_acc)
        fold_val_acc.append(val_acc)
        fold_val_sens.append(val_sens)
        fold_val_spec.append(val_spec)

    # === Mean over folds
    avg_train_bal_acc = np.mean(fold_train_bal_acc)
    avg_train_acc = np.mean(fold_train_acc)
    avg_train_sens = np.mean(fold_train_sens)
    avg_train_spec = np.mean(fold_train_spec)

    avg_val_bal_acc = np.mean(fold_val_bal_acc)
    avg_val_acc = np.mean(fold_val_acc)
    avg_val_sens = np.mean(fold_val_sens)
    avg_val_spec = np.mean(fold_val_spec)

    print(f"\nMean over folds for C={C}, gamma={gamma}:")
    print(f"  Train - BalAcc: {avg_train_bal_acc:.4f} Acc: {avg_train_acc:.4f} Sens: {avg_train_sens:.4f} Spec: {avg_train_spec:.4f}")
    print(f"  Val   - BalAcc: {avg_val_bal_acc:.4f} Acc: {avg_val_acc:.4f} Sens: {avg_val_sens:.4f} Spec: {avg_val_spec:.4f}\n")

    # --- Best tracking
    if avg_val_bal_acc > best_score:
        best_score = avg_val_bal_acc
        best_params = (C, gamma)
        best_metrics = {
            "Balanced Accuracy": avg_val_bal_acc,
            "Accuracy": avg_val_acc,
            "Sensitivity": avg_val_sens,
            "Specificity": avg_val_spec,
            "Train Balanced Accuracy": avg_train_bal_acc,
            "Train Accuracy": avg_train_acc,
            "Train Sensitivity": avg_train_sens,
            "Train Specificity": avg_train_spec,
        }

print("\n✅ Best Grid Search Result:")
print(f"Best C: {best_params[0]}")
print(f"Best gamma: {best_params[1]}")
print(f"Best Validation Balanced Accuracy: {best_metrics['Balanced Accuracy']:.4f}")
print(f"Best Validation Accuracy:          {best_metrics['Accuracy']:.4f}")
print(f"Best Validation Sensitivity:       {best_metrics['Sensitivity']:.4f}")
print(f"Best Validation Specificity:       {best_metrics['Specificity']:.4f}")
print(f"Best Training Balanced Accuracy:   {best_metrics['Train Balanced Accuracy']:.4f}")
print(f"Best Training Accuracy:            {best_metrics['Train Accuracy']:.4f}")
print(f"Best Training Sensitivity:         {best_metrics['Train Sensitivity']:.4f}")
print(f"Best Training Specificity:         {best_metrics['Train Specificity']:.4f}")



Running Grid Search over C and gamma with 5-Fold Stratified CV...



  0%|          | 1/441 [02:12<16:13:42, 132.78s/it]


Mean over folds for C=0.0009765625, gamma=0.0009765625:
  Train - BalAcc: 0.5000 Acc: 0.4177 Sens: 0.6000 Spec: 0.4000
  Val   - BalAcc: 0.5000 Acc: 0.4181 Sens: 0.6000 Spec: 0.4000



  0%|          | 2/441 [04:26<16:17:08, 133.55s/it]


Mean over folds for C=0.0009765625, gamma=0.001953125:
  Train - BalAcc: 0.5000 Acc: 0.4177 Sens: 0.6000 Spec: 0.4000
  Val   - BalAcc: 0.5000 Acc: 0.4181 Sens: 0.6000 Spec: 0.4000



...


Mean over folds for C=1024, gamma=256:
  Train - BalAcc: 1.0000 Acc: 1.0000 Sens: 1.0000 Spec: 1.0000
  Val   - BalAcc: 0.4999 Acc: 0.9109 Sens: 0.0000 Spec: 0.9999



100%|█████████▉| 440/441 [14:04:38<02:22, 142.05s/it]


Mean over folds for C=1024, gamma=512:
  Train - BalAcc: 1.0000 Acc: 1.0000 Sens: 1.0000 Spec: 1.0000
  Val   - BalAcc: 0.4999 Acc: 0.9109 Sens: 0.0000 Spec: 0.9999



100%|██████████| 441/441 [14:06:59<00:00, 115.24s/it]


Mean over folds for C=1024, gamma=1024:
  Train - BalAcc: 1.0000 Acc: 1.0000 Sens: 1.0000 Spec: 1.0000
  Val   - BalAcc: 0.4999 Acc: 0.9109 Sens: 0.0000 Spec: 0.9999


✅ Best Grid Search Result:
Best C: 0.0078125
Best gamma: 0.25
Best Validation Balanced Accuracy: 0.7996
Best Validation Accuracy:          0.8952
Best Validation Sensitivity:       0.6833
Best Validation Specificity:       0.9159
Best Training Balanced Accuracy:   0.7996
Best Training Accuracy:            0.8952
Best Training Sensitivity:         0.6834
Best Training Specificity:         0.9159


##### Test

In [ ]:
# 取得最佳參數
best_C = 0.0078125
best_gamma = 0.25
final_model = SVC(
    C=best_C,
    gamma=best_gamma,
    kernel='rbf',
    probability=True,
    class_weight='balanced',
    random_state=seed
)

final_model.fit(x, y)


# 預測
y_test_pred = final_model.predict(X_test.values)                  # 分類標籤

# 混淆矩陣
from sklearn.metrics import confusion_matrix, roc_auc_score

cm = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm.ravel()

# 指標計算
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
accuracy = (tp + tn) / (tp + tn + fp + fn)
balanced_acc = (sensitivity + specificity) / 2
auc = roc_auc_score(y_test, y_test_proba)
f1 = f1_score(y_test, y_test_pred, zero_division=0)

# 轉成 DataFrame (Wikipedia 標準)
confusion_df = pd.DataFrame(
    data=[[tp, fn], [fp, tn]],
    index=["Actual Positive", "Actual Negative"],
    columns=["Predicted Positive", "Predicted Negative"]
)

print("\n✅ Confusion Matrix (DataFrame, Wikipedia format):")
print(confusion_df)

# 顯示結果
print(f"Sensitivity (Recall, TPR): {sensitivity:.4f}")
print(f"Specificity (TNR):         {specificity:.4f}")
print(f"Accuracy:                  {accuracy:.4f}")
print(f"Balanced Accuracy:         {balanced_acc:.4f}")
print(f"AUC:                       {auc:.4f}")
print(f"F1 Score:         {f1:.4f}")


✅ Confusion Matrix (DataFrame, Wikipedia format):
                 Predicted Positive  Predicted Negative
Actual Positive                 112                  63
Actual Negative                 139                1648
Sensitivity (Recall, TPR): 0.6400
Specificity (TNR):         0.9222
Accuracy:                  0.8970
Balanced Accuracy:         0.7811
AUC:                       0.7916
F1 Score:         0.5258


#### XGboost


In [ ]:
from collections import Counter

# 先計算 scale_pos_weight
counter = Counter(y)
print(counter)

n_pos = counter[1]
n_neg = counter[0]
scale_pos_weight = n_neg / n_pos
print(f"scale_pos_weight = {scale_pos_weight:.4f}")

Counter({np.int64(0): 7149, np.int64(1): 698})
scale_pos_weight = 10.2421


In [ ]:
!pip install optuna==3.6.1   # 最新穩定版本

Found existing installation: optuna 4.4.0
Uninstalling optuna-4.4.0:
  Successfully uninstalled optuna-4.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.8 MB/s eta 0:00:00


In [ ]:
import optuna

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

def objective(trial):
    params = {
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 0.9),
        "gamma": trial.suggest_float("gamma", 0.0, 0.5),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.16),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0)
    }

    fold_train_bal_acc, fold_train_acc, fold_train_sens, fold_train_spec = [], [], [], []
    fold_val_bal_acc, fold_val_acc, fold_val_sens, fold_val_spec = [], [], [], []

    for train_idx, val_idx in cv.split(x, y):
        x_train_fold, y_train_fold = x[train_idx], y[train_idx]
        x_val_fold, y_val_fold = x[val_idx], y[val_idx]

        clf = xgb.XGBClassifier(
            **params,
            eval_metric="logloss",
            tree_method="hist",
            device="cuda",
            scale_pos_weight=scale_pos_weight,
            random_state=seed
        )
        clf.fit(x_train_fold, y_train_fold)

        # 訓練集預測
        y_train_pred = clf.predict(x_train_fold)
        tn, fp, fn, tp = confusion_matrix(y_train_fold, y_train_pred).ravel()
        train_sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        train_spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        train_acc = (tp + tn) / (tp + tn + fp + fn)
        train_bal_acc = (train_sens + train_spec) / 2

        fold_train_bal_acc.append(train_bal_acc)
        fold_train_acc.append(train_acc)
        fold_train_sens.append(train_sens)
        fold_train_spec.append(train_spec)

        # 驗證集預測
        y_val_pred = clf.predict(x_val_fold)
        tn, fp, fn, tp = confusion_matrix(y_val_fold, y_val_pred).ravel()
        val_sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        val_spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        val_acc = (tp + tn) / (tp + tn + fp + fn)
        val_bal_acc = (val_sens + val_spec) / 2

        fold_val_bal_acc.append(val_bal_acc)
        fold_val_acc.append(val_acc)
        fold_val_sens.append(val_sens)
        fold_val_spec.append(val_spec)

    # 平均
    avg_train_bal_acc = np.mean(fold_train_bal_acc)
    avg_train_acc = np.mean(fold_train_acc)
    avg_train_sens = np.mean(fold_train_sens)
    avg_train_spec = np.mean(fold_train_spec)
    avg_val_bal_acc = np.mean(fold_val_bal_acc)
    avg_val_acc = np.mean(fold_val_acc)
    avg_val_sens = np.mean(fold_val_sens)
    avg_val_spec = np.mean(fold_val_spec)

    print(f"\nTrial {trial.number+1:03d} Params: {params}")
    print(f"  Train - Balanced Acc: {avg_train_bal_acc:.4f} | Acc: {avg_train_acc:.4f} "
          f"| Sens: {avg_train_sens:.4f} | Spec: {avg_train_spec:.4f}")
    print(f"  Val   - Balanced Acc: {avg_val_bal_acc:.4f} | Acc: {avg_val_acc:.4f} "
          f"| Sens: {avg_val_sens:.4f} | Spec: {avg_val_spec:.4f}")

    # 儲存 trial 結果
    trial.set_user_attr("train_bal_acc", avg_train_bal_acc)
    trial.set_user_attr("train_acc", avg_train_acc)
    trial.set_user_attr("train_sens", avg_train_sens)
    trial.set_user_attr("train_spec", avg_train_spec)
    trial.set_user_attr("val_bal_acc", avg_val_bal_acc)
    trial.set_user_attr("val_acc", avg_val_acc)
    trial.set_user_attr("val_sens", avg_val_sens)
    trial.set_user_attr("val_spec", avg_val_spec)

    return avg_val_bal_acc

# === 執行 Optuna ===
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=2000)

# === 最佳 trial ===
best = study.best_trial

print("\nBest trial:")
print(f"  Params: {best.params}")
print(f"  Value: {best.value:.4f}")
print(f"  Train - Balanced Acc: {best.user_attrs['train_bal_acc']:.4f} "
      f"| Acc: {best.user_attrs['train_acc']:.4f} "
      f"| Sens: {best.user_attrs['train_sens']:.4f} "
      f"| Spec: {best.user_attrs['train_spec']:.4f}")
print(f"  Val   - Balanced Acc: {best.user_attrs['val_bal_acc']:.4f} "
      f"| Acc: {best.user_attrs['val_acc']:.4f} "
      f"| Sens: {best.user_attrs['val_sens']:.4f} "
      f"| Spec: {best.user_attrs['val_spec']:.4f}")



[I 2025-08-02 14:26:51,398] A new study created in memory with name: no-name-b87f1d73-06d3-4048-84d9-d5db7755544c
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:729: UserWarning: [14:26:52] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)
[I 2025-08-02 14:26:56,867] Trial 0 finished with value: 0.7213734250706325 and parameters: {'colsample_bytree': 0.7567122059443789, 'gamma': 0.4327153634253259, 'learning_rate': 0.04809176851599164, 'max_depth': 9, 'n_estimators': 332, 'subsample': 0.7817398028403169}. Best is trial 0 with value: 0.7213734250706325.



Trial 001 Params: {'colsample_bytree': 0.7567122059443789, 'gamma': 0.4327153634253259, 'learning_rate': 0.04809176851599164, 'max_depth': 9, 'n_estimators': 332, 'subsample': 0.7817398028403169}
  Train - Balanced Acc: 0.9994 | Acc: 0.9990 | Sens: 1.0000 | Spec: 0.9989
  Val   - Balanced Acc: 0.7214 | Acc: 0.9164 | Sens: 0.4842 | Spec: 0.9586


[I 2025-08-02 14:26:59,192] Trial 1 finished with value: 0.712077815579711 and parameters: {'colsample_bytree': 0.7564559263140588, 'gamma': 0.47992392584179605, 'learning_rate': 0.12617826786935865, 'max_depth': 7, 'n_estimators': 228, 'subsample': 0.8035110667402089}. Best is trial 0 with value: 0.7213734250706325.



Trial 002 Params: {'colsample_bytree': 0.7564559263140588, 'gamma': 0.47992392584179605, 'learning_rate': 0.12617826786935865, 'max_depth': 7, 'n_estimators': 228, 'subsample': 0.8035110667402089}
  Train - Balanced Acc: 0.9994 | Acc: 0.9990 | Sens: 1.0000 | Spec: 0.9989
  Val   - Balanced Acc: 0.7121 | Acc: 0.9159 | Sens: 0.4642 | Spec: 0.9600


[I 2025-08-02 14:27:04,052] Trial 2 finished with value: 0.7101690080338177 and parameters: {'colsample_bytree': 0.7574075706819243, 'gamma': 0.14074105446747404, 'learning_rate': 0.07332615968447614, 'max_depth': 9, 'n_estimators': 392, 'subsample': 0.9364322199859054}. Best is trial 0 with value: 0.7213734250706325.


...


Trial 1998 Params: {'colsample_bytree': 0.892951221844559, 'gamma': 0.07230624637751185, 'learning_rate': 0.03887968285692978, 'max_depth': 3, 'n_estimators': 131, 'subsample': 0.8054050099342878}
  Train - Balanced Acc: 0.8266 | Acc: 0.8891 | Sens: 0.7507 | Spec: 0.9026
  Val   - Balanced Acc: 0.8012 | Acc: 0.8817 | Sens: 0.7033 | Spec: 0.8991


[I 2025-08-02 15:12:03,086] Trial 1998 finished with value: 0.7962116663084287 and parameters: {'colsample_bytree': 0.8957971323845308, 'gamma': 0.056208313812297496, 'learning_rate': 0.042017813395274724, 'max_depth': 4, 'n_estimators': 145, 'subsample': 0.828443988905996}. Best is trial 1545 with value: 0.8046634838345599.



Trial 1999 Params: {'colsample_bytree': 0.8957971323845308, 'gamma': 0.056208313812297496, 'learning_rate': 0.042017813395274724, 'max_depth': 4, 'n_estimators': 145, 'subsample': 0.828443988905996}
  Train - Balanced Acc: 0.8625 | Acc: 0.9111 | Sens: 0.8034 | Spec: 0.9216
  Val   - Balanced Acc: 0.7962 | Acc: 0.8903 | Sens: 0.6818 | Spec: 0.9106


[I 2025-08-02 15:12:03,932] Trial 1999 finished with value: 0.8006819785578136 and parameters: {'colsample_bytree': 0.8917080711376242, 'gamma': 0.043352445199477826, 'learning_rate': 0.03331929700864737, 'max_depth': 3, 'n_estimators': 144, 'subsample': 0.8214375160242642}. Best is trial 1545 with value: 0.8046634838345599.



Trial 2000 Params: {'colsample_bytree': 0.8917080711376242, 'gamma': 0.043352445199477826, 'learning_rate': 0.03331929700864737, 'max_depth': 3, 'n_estimators': 144, 'subsample': 0.8214375160242642}
  Train - Balanced Acc: 0.8244 | Acc: 0.8879 | Sens: 0.7471 | Spec: 0.9017
  Val   - Balanced Acc: 0.8007 | Acc: 0.8796 | Sens: 0.7047 | Spec: 0.8966

Best trial:
  Params: {'colsample_bytree': 0.8831528055561403, 'gamma': 0.021020125361070625, 'learning_rate': 0.03754890989284685, 'max_depth': 3, 'n_estimators': 181, 'subsample': 0.8246355080259108}
  Value: 0.8047
  Train - Balanced Acc: 0.8336 | Acc: 0.8924 | Sens: 0.7622 | Spec: 0.9051
  Val   - Balanced Acc: 0.8047 | Acc: 0.8821 | Sens: 0.7105 | Spec: 0.8989


##### Test

In [ ]:
print("\nTraining final model on full training data with best parameters...")
final_model = xgb.XGBClassifier(
    **best.params,
    eval_metric="logloss",
    tree_method="hist",
    device="cuda",
    scale_pos_weight=scale_pos_weight,
    random_state=seed
)
final_model.fit(x, y)


# 預測 Test Set
y_test_proba = final_model.predict_proba(X_test.values)[:, 1]
y_test_pred = (y_test_proba >= 0.5).astype(int)

# 混淆矩陣
from sklearn.metrics import confusion_matrix, roc_auc_score

cm = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm.ravel()

# 指標計算
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
accuracy = (tp + tn) / (tp + tn + fp + fn)
balanced_acc = (sensitivity + specificity) / 2
auc = roc_auc_score(y_test, y_test_proba)
f1 = f1_score(y_test, y_test_pred, zero_division=0)

# 轉成 DataFrame (Wikipedia 標準)
confusion_df = pd.DataFrame(
    data=[[tp, fn], [fp, tn]],
    index=["Actual Positive", "Actual Negative"],
    columns=["Predicted Positive", "Predicted Negative"]
)

print("\n✅ Confusion Matrix (DataFrame, Wikipedia format):")
print(confusion_df)

# 顯示結果
print(f"Sensitivity (Recall, TPR): {sensitivity:.4f}")
print(f"Specificity (TNR):         {specificity:.4f}")
print(f"Accuracy:                  {accuracy:.4f}")
print(f"Balanced Accuracy:         {balanced_acc:.4f}")
print(f"AUC:                       {auc:.4f}")
print(f"F1 Score:                  {f1:.4f}")


Training final model on full training data with best parameters...

✅ Confusion Matrix (DataFrame, Wikipedia format):
                 Predicted Positive  Predicted Negative
Actual Positive                 121                  54
Actual Negative                 178                1609
Sensitivity (Recall, TPR): 0.6914
Specificity (TNR):         0.9004
Accuracy:                  0.8818
Balanced Accuracy:         0.7959
AUC:                       0.8658
F1 Score:                  0.5105
